# Part 02 Feature Engineering

# 0.0. Imports

In [1]:
# Data manipulation
import pandas as pd
import numpy as np

# Statistics
from scipy import stats
from pandas_profiling import ProfileReport

# Graphs
import matplotlib.pyplot as plt
import seaborn as sns

# Load images
from IPython.display import Image

# Warning
import warnings
warnings.filterwarnings( 'ignore' )

# 3.0. Feature Engineering
Theoretically, the difference between a previous transaction and a subsequent one would be equal to the total transferred:

>amount = oldbalanceOrg - newbalanceOrig

The same could be said for Destinations:

>newbalanceDest = amount + oldbalanceDest

In [2]:
# Load dataset
feature_dtypes = {'step': 'int16',
                  'type': 'category',
                  'amount': 'float32',
                  'nameOrig': 'object',
                  'oldbalanceOrg': 'float32',
                  'newbalanceOrig': 'float32',
                  'nameDest': 'object',
                  'oldbalanceDest': 'float32',
                  'newbalanceDest': 'float32',
                  'isFraud': 'uint8',
                  'isFlaggedFraud': 'uint8'}

df1 = pd.read_csv('dataset\df1.csv', dtype=feature_dtypes)

,step,type,amount,name_orig,oldbalance_orig,newbalance_orig,name_dest,oldbalance_dest,newbalance_dest,is_fraud,is_flagged_fraud
0,1,PAYMENT,9839.639648,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.280029,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.000000,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.000000,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.139648,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [3]:
# Origin balance
df1['error_orig'] = df1.oldbalance_orig - df1.newbalance_orig - df1.amount

# Destination balance
df1['error_dest'] = df1.oldbalance_dest + df1.amount - df1.newbalance_dest

A new feature classifying the type of customers.

In [4]:
# Customer type for name origin
# df1['orig_type'] = df1['name_orig'].str.replace(r'[0-9]', regex=True, repl='')

# Customer type for name destination
df1['dest_type'] = df1['name_dest'].str.replace(r'[0-9]', regex=True, repl='')

The `orig_type` feature has only one customer type, that column will be removed.

In [5]:
df1.dest_type.value_counts()

C    4211125
M    2151495
Name: dest_type, dtype: int64

Feature day will be created in order to ease the analysis.

In [6]:
# Day column
df1['days'] = df1.step.apply(lambda x: np.ceil(x/24).astype('int'))

In [11]:
# Saving dataset
df1.to_csv('dataset\df2.csv', index=False)

In [8]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 15 columns):
 #   Column            Dtype   
---  ------            -----   
 0   step              int16   
 1   type              category
 2   amount            float32 
 3   name_orig         object  
 4   oldbalance_orig   float64 
 5   newbalance_orig   float64 
 6   name_dest         object  
 7   oldbalance_dest   float64 
 8   newbalance_dest   float64 
 9   is_fraud          int64   
 10  is_flagged_fraud  int64   
 11  error_orig        float64 
 12  error_dest        float64 
 13  dest_type         object  
 14  days              int64   
dtypes: category(1), float32(1), float64(6), int16(1), int64(3), object(3)
memory usage: 625.0+ MB


In [10]:
df1.describe()

,step,amount,oldbalance_orig,newbalance_orig,oldbalance_dest,newbalance_dest,is_fraud,is_flagged_fraud,error_orig,error_dest,days
count,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06
mean,2.433972e+02,1.779883e+05,8.338831e+05,8.551137e+05,1.100702e+06,1.224996e+06,1.290820e-03,2.514687e-06,-2.010925e+05,5.556717e+04,1.049191e+01
std,1.423320e+02,5.991358e+05,2.888243e+06,2.924049e+06,3.399180e+06,3.674129e+06,3.590480e-02,1.585775e-03,6.066505e+05,4.415288e+05,5.921812e+00
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-9.244552e+07,-7.588572e+07,1.000000e+00
25%,1.560000e+02,1.338957e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-2.496411e+05,6.250000e-04,7.000000e+00
50%,2.390000e+02,7.487194e+04,1.420800e+04,0.000000e+00,1.327057e+05,2.146614e+05,0.000000e+00,0.000000e+00,-6.867725e+04,3.500490e+03,1.000000e+01
75%,3.350000e+02,2.087215e+05,1.073152e+05,1.442584e+05,9.430367e+05,1.111909e+06,0.000000e+00,0.000000e+00,-2.954197e+03,2.935302e+04,1.400000e+01
max,7.430000e+02,9.244552e+07,5.958504e+07,4.958504e+07,3.560159e+08,3.561793e+08,1.000000e+00,1.000000e+00,4.240234e+00,1.319123e+07,3.100000e+01
